# Text Encoding

Trong quá trình xử lý dữ liệu thực tế, rất nhiều thuộc tính (cột) xuất hiện dưới dạng văn bản như tên danh mục, trạng thái, mô tả, bình luận,… Tuy nhiên, các mô hình Machine Learning không thể làm việc trực tiếp với dữ liệu dạng text. Vì vậy, bước mã hóa dữ liệu (Text Encoding) là một phần quan trọng trong giai đoạn tiền xử lý dữ liệu (data preprocessing).

Notebook này trình bày các kỹ thuật phổ biến để chuyển đổi văn bản thành dạng số, giúp mô hình có thể hiểu và học từ dữ liệu:

1. One-Hot Encoding – biến mỗi giá trị phân loại thành một vector nhị phân
2. Target Encoding – mã hoá theo mục tiêu

# 0. Nguồn dữ liệu

In [1]:
import pandas as pd
import unidecode

df = pd.read_csv('../data/interim/combined_interim.csv')

fields = ['origin', 'status', 'brand', 'series', 'fuel_type', 'style']

# for col in fields:
#     print("\n" + "="*50)
#     print(f"THỐNG KÊ FIELD: {col.upper()}")
#     print("="*50)
#
#     vc = df[col].value_counts(dropna=False)
#
#     print(f"➡ Số giá trị unique: {df[col].nunique(dropna=False)}")
#
#     print(vc.to_string())
#     print("="*50 + "\n")


# 1. One-hot field
Một số trường dữ liệu có loại dữ liệu thuộc một tập hữu hạn trong khoảng dưới 30 loại (có thể chọn đến 50) sẽ được encoding theo one-hot

### 1.1. status

In [2]:
status_mapping = {
    'đã sử dụng': 'used',
    'xe đã dùng': 'used',
    'xe mới': 'new',
    'mới': 'new'
}

df['status'] = (
    df['status']
    .astype(str)          # chuyển thành chuỗi
    .str.lower()          # lowercase để map dễ dàng
    .str.strip()          # loại bỏ khoảng trắng đầu/cuối
    .replace(status_mapping)  # gom nhãn
)

print(df['status'].value_counts())

status
used    12437
new      2491
Name: count, dtype: int64


### 1.2. fuel

In [3]:
fuel_mapping = {
    'xăng': 'gasoline',
    'dầu': 'diesel',
    'điện': 'electric',
    'hybrid': 'hybrid'
}

df['fuel_type'] = (
    df['fuel_type']
    .astype(str)
    .str.lower()
    .str.strip()
    .replace(fuel_mapping)
)

print(df['fuel_type'].value_counts())

fuel_type
gasoline          11402
diesel             2209
electric            841
hybrid              443
động cơ hybrid       33
Name: count, dtype: int64


### 1.3. origin

In [4]:
# Lấy danh sách các giá trị origin
origins = df['origin'].unique()

# Tạo mapping: chuẩn hóa tên
origin_mapping = {orig: unidecode.unidecode(orig).lower().replace(' ', '_') for orig in origins}

# Áp dụng mapping
df['origin'] = df['origin'].map(origin_mapping)

print(df['origin'].value_counts())


origin
trong_nuoc       5159
nhap_khau        4841
viet_nam         1989
dang_cap_nhat    1095
thai_lan          426
han_quoc          375
nhat_ban          327
nuoc_khac         268
duc               148
my                147
trung_quoc         83
an_do              57
dai_loan           13
Name: count, dtype: int64


### 1.4. brand

In [5]:
total_samples = len(df)
threshold = 0.005 * total_samples  # 0.5% tổng mẫu

# Đếm tần suất từng brand
brand_counts = df['brand'].value_counts()

# Gom các brand ít xuất hiện thành 'other'
df['brand'] = df['brand'].apply(lambda x: x if brand_counts[x] >= threshold else 'other')


### 1.5. gear

In [6]:
gear_mapping = {
    'số sàn': "mt",
    'tự động': 'at',
    'automatic': 'at',
    'manual': 'mt'
}

df['gear'] = (
    df['gear']
    .astype(str)          # chuyển thành chuỗi
    .str.lower()          # lowercase để map dễ dàng
    .str.strip()          # loại bỏ khoảng trắng đầu/cuối
    .replace(gear_mapping)  # gom nhãn
)

print(df['gear'].value_counts())

gear
at             7078
nan            5851
mt             1960
bán tự động      21
5                17
4                 1
Name: count, dtype: int64


### 1.6 style

In [7]:

style_mapping = {
    # SUV
    'SUV / Cross over': 'SUV',
    'SUV': 'SUV',

    # Sedan
    'Sedan': 'Sedan',

    # Hatchback
    'Hatchback': 'Hatchback',

    # Minivan / Van
    'Minivan (MPV)': 'Minivan',
    'Van/Minivan': 'Minivan',
    'Van': 'Minivan',

    # Pickup / Bán tải
    'Bán tải / Pickup': 'Pickup',
    'Pick-up (bán tải)': 'Pickup',

    # Coupe
    'Coupe': 'Coupe',
    'Coupe (2 cửa)': 'Coupe',

    # Convertible
    'Convertible/Cabriolet': 'Convertible',

    # Kiểu dáng khác / unknown
    'Kiểu dáng khác': 'other',
    None: 'unknown',  # xử lý NaN
    'NaN': 'unknown'
}

# Áp dụng mapping
df['style'] = df['style'].map(style_mapping).fillna(df['style'])
# Chuẩn hóa tên: lowercase + no dấu + replace space bằng _
df['style'] = df['style'].apply(lambda x: unidecode.unidecode(str(x)).lower().replace(' ', '_'))

# Kiểm tra kết quả
print(df['style'].value_counts())

style
suv            6247
sedan          3763
minivan        1203
hatchback      1088
nan             840
crossover       759
pickup          753
coupe            93
other            81
convertible      61
truck            32
mui_tran          4
wagon             4
Name: count, dtype: int64


# 2. Target Encoding

# 3. Lưu thay đổi

In [8]:
# === One-Hot Encoding ===
df = pd.get_dummies(df,
                    columns=['status', 'fuel_type', 'origin', 'style', 'brand','gear'],
                    prefix=['status', 'fuel', 'origin', 'style', 'brand','gear'])

# Chuyển True/False sang 0/1 cho các cột one-hot
dummy_cols = [col for col in df.columns if col.startswith('status_')
                                      or col.startswith('fuel_')
                                      or col.startswith('origin_')
                                      or col.startswith('style_')
                                      or col.startswith('brand_')
                                      or col.startswith('gear_')]
df[dummy_cols] = df[dummy_cols].astype(int)

# === Giữ nguyên kiểu dữ liệu các cột số ===
numeric_cols = ['price', 'odometer', 'seats', 'year']

for col in numeric_cols:
    if col in df.columns:
        # price luôn int64
        if col == 'price':
            df[col] = df[col].astype('int64')
        # year: Int64 nếu có NaN, int64 nếu không NaN
        elif col == 'year':
            df[col] = df[col].astype('Int64')
        # odometer: Int64 nếu có NaN, int64 nếu không
        elif col == 'odometer':
            if df[col].notna().all():
                df[col] = df[col].astype('int64')
            else:
                df[col] = df[col].astype('Int64')
        # seats: Int64 nếu có NaN, int64 nếu không
        elif col == 'seats':
            if df[col].notna().all():
                df[col] = df[col].astype('int64')
            else:
                df[col] = df[col].astype('Int64')

# === Xuất CSV ===
csv_file = '../data/interim/encoding_interim.csv'
df.to_csv(csv_file, index=False, encoding='utf-8-sig')

# === Kết quả ===
print("\n✓ Đã áp dụng One-Hot Encoding")
print(f"Số cột sau One-Hot Encoding: {df.shape[1]}")
print(f"\nCác cột mới:")
print([col for col in df.columns if col.startswith('status_') or col.startswith('fuel_')])
print("\nDữ liệu sau encoding:")
print(df.head())



✓ Đã áp dụng One-Hot Encoding
Số cột sau One-Hot Encoding: 71

Các cột mới:
['status_new', 'status_used', 'fuel_diesel', 'fuel_electric', 'fuel_gasoline', 'fuel_hybrid', 'fuel_động cơ hybrid']

Dữ liệu sau encoding:
       price  odometer   series  year  seats  \
0  320000000     88000       X5  2007      8   
1  108000000    135000  E Class  2001      5   
2  365000000    275000  Transit  2017     16   
3  339000000     60150    Cruze  2016      5   
4  380000000     15000    Camry  2009      4   

                                     location  engine  status_new  \
0            Phường 1, Quận 4, Tp Hồ Chí Minh     NaN           0   
1            Phường 4, Quận 8, Tp Hồ Chí Minh     NaN           0   
2     Phường Thốt Nốt, Quận Thốt Nốt, Cần Thơ     NaN           0   
3      Phường Ba Láng, Quận Cái Răng, Cần Thơ     NaN           0   
4  Phường Mỹ Đình 1, Quận Nam Từ Liêm, Hà Nội     NaN           0   

   status_used  fuel_diesel  ...  brand_VinFast  brand_Volkswagen  \
0         